# 5.3. Przygotowanie danych do klasyfikacji

Mamy już wyrobione pewne intuicje co do naszego zbioru danych. Nie jest on jednak idealnie dostosowany do modelowania i musimy przeprowadzić pewne transformacje, aby niejako wyciągnąć dodatkowe informacje.

In [1]:
import pandas as pd

In [2]:
credit_cards_df = pd.read_excel(
    "../data/credit-cards/default-of-credit-card-clients.xls",
    index_col=0, skiprows=1
).rename(
    columns={"PAY_0": "PAY_1", 
             "default payment next month": "DEFAULT"}
)

credit_cards_df.sample(n=5).T

ID,16456,27225,14571,11136,17895
LIMIT_BAL,100000,80000,400000,90000,200000
SEX,2,1,2,2,1
EDUCATION,2,3,1,1,2
MARRIAGE,1,2,2,2,2
AGE,52,34,41,27,27
PAY_1,2,0,-1,0,2
PAY_2,2,0,-1,0,0
PAY_3,2,0,-1,0,0
PAY_4,2,0,0,0,0
PAY_5,0,0,-1,0,0


## Agregacja powiązanych zmiennych

Wszystkie kolumny *PAY_\** zawierają jednocześnie informacje o długości opóźnienia w spłacie, jak i sam fakt że dana osoba nie korzystała z karty w tym miesiącu bądź też spłaciła w terminie. Fakt długości spóźnienia warto odnotować w postaci zmiennej numerycznej, jednak historię wystąpienia jakiegokolwiek opóźnienia w przeszłości, jest równie ciekawy i można np. zanotować jak wiele razy dana osoba spóźniła się kiedykolwiek ze spłatą.

In [3]:
def count_late_payments(row):
    count = 0
    columns = ["PAY_1", "PAY_2", "PAY_3", 
               "PAY_4", "PAY_5", "PAY_6"]
    for column in columns:
        if row[column] > 0:
            count += 1
    return count

In [4]:
credit_cards_df["PAY_OVERDUE_COUNT"] = credit_cards_df \
    .apply(count_late_payments, axis="columns")

Nasze dane zawierają historię ewentualnych problemów ze spłatą, więc może warto wnioskować, że z problemem niewypłacalności mogą zderzyć się klienci, których sytuacja finansowa znacząco się pogorszyła. Skorzystajmy więc z ważonej sumy, dzięki której ostatnie zdarzenia będą miały większy wpływ niż te, które wystąpiły dawniej.

In [5]:
def weighted_payment_history(row):
    weighted_sum = 0.0
    columns = ["PAY_1", "PAY_2", "PAY_3", 
               "PAY_4", "PAY_5", "PAY_6"]
    for i, column in enumerate(columns):
        weighted_sum += row[column] / (i + 1)
    return weighted_sum

In [6]:
credit_cards_df["WEIGHTED_PAYMENT_HISTORY"] = credit_cards_df \
    .apply(weighted_payment_history, axis="columns")

Korelacja wszystkich zmiennych *BILL_\** oraz *PAY_\** jest bardzo niska. Spróbujmy zatem zagregować je, aby zostawić sobie jedynie średnie kwoty.

In [7]:
import numpy as np

In [8]:
def multiple_col_avg(row, columns):
    return np.mean(row[columns])

In [9]:
credit_cards_df["AVG_BILL_AMT"] = credit_cards_df \
    .apply(multiple_col_avg, axis="columns", 
           args=(["BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                  "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"], ))

In [10]:
credit_cards_df["AVG_PAY_AMT"] = credit_cards_df \
    .apply(multiple_col_avg, axis="columns", 
           args=(["PAY_AMT1", "PAY_AMT2", "PAY_AMT3",
                  "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"], ))

## Analiza typów danych

Sprawdźmy teraz, jak pandas reprezentuje poszczególne kolumny pod kątem ich typów.

In [11]:
credit_cards_df.dtypes

LIMIT_BAL                     int64
SEX                           int64
EDUCATION                     int64
MARRIAGE                      int64
AGE                           int64
PAY_1                         int64
PAY_2                         int64
PAY_3                         int64
PAY_4                         int64
PAY_5                         int64
PAY_6                         int64
BILL_AMT1                     int64
BILL_AMT2                     int64
BILL_AMT3                     int64
BILL_AMT4                     int64
BILL_AMT5                     int64
BILL_AMT6                     int64
PAY_AMT1                      int64
PAY_AMT2                      int64
PAY_AMT3                      int64
PAY_AMT4                      int64
PAY_AMT5                      int64
PAY_AMT6                      int64
DEFAULT                       int64
PAY_OVERDUE_COUNT             int64
WEIGHTED_PAYMENT_HISTORY    float64
AVG_BILL_AMT                float64
AVG_PAY_AMT                 

Zmienne płci, wykształcenia oraz stanu cywilnego nie powinny być traktowane jak zmienne numeryczne, ponieważ nie istnieje pomiędzy nimi porządek, bądź też nie da się wykonywać podstawowych operacji.

In [12]:
credit_cards_df = credit_cards_df.astype({
    "SEX": "category",
    "EDUCATION": "category",
    "MARRIAGE": "category"
})

Skorzystajmy teraz z one-hot encoding do zamiany zmiennych z kategoriami na reprezentację w osobnych kolumnach.

In [13]:
credit_cards_df = pd.get_dummies(credit_cards_df, 
                                 drop_first=True)
credit_cards_df.sample(n=5).T

ID,28353,26888,18403,23177,26000
LIMIT_BAL,250000.000000,360000.000000,50000.000000,290000.000000,20000.000000
AGE,36.000000,36.000000,23.000000,35.000000,33.000000
PAY_1,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_2,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_3,0.000000,0.000000,0.000000,-1.000000,0.000000
PAY_4,0.000000,0.000000,2.000000,-1.000000,0.000000
PAY_5,0.000000,0.000000,0.000000,-1.000000,0.000000
PAY_6,0.000000,0.000000,0.000000,0.000000,0.000000
BILL_AMT1,71012.000000,338992.000000,32206.000000,32485.000000,10001.000000
BILL_AMT2,60816.000000,339254.000000,65339.000000,21080.000000,20844.000000


In [14]:
credit_cards_df.corr()["DEFAULT"]

LIMIT_BAL                  -0.153520
AGE                         0.013890
PAY_1                       0.324794
PAY_2                       0.263551
PAY_3                       0.235253
PAY_4                       0.216614
PAY_5                       0.204149
PAY_6                       0.186866
BILL_AMT1                  -0.019644
BILL_AMT2                  -0.014193
BILL_AMT3                  -0.014076
BILL_AMT4                  -0.010156
BILL_AMT5                  -0.006760
BILL_AMT6                  -0.005372
PAY_AMT1                   -0.072929
PAY_AMT2                   -0.058579
PAY_AMT3                   -0.056250
PAY_AMT4                   -0.056827
PAY_AMT5                   -0.055124
PAY_AMT6                   -0.053183
DEFAULT                     1.000000
PAY_OVERDUE_COUNT           0.398394
WEIGHTED_PAYMENT_HISTORY    0.317726
AVG_BILL_AMT               -0.012691
AVG_PAY_AMT                -0.102354
SEX_2                      -0.039961
EDUCATION_1                -0.051328
E

Przypomnijmy sobie jak kodowane były zmienne kategoryczne:
- **SEX** - płeć (1 = mężczyzna, 2 = kobieta)
- **EDUCATION** - wykształcenie, wartość od 1 do 6 (1 = wyższe pełne, 2 = wyższe (1. stopień), 3 = średnie, 4 = inne, 5 i 6 = nieznane)
- **MARRIAGE** - stan cywilny (1 = zamężna/żonaty, 2 = panna/kawaler, 3 = inny)

## Zapis stworzonego zbioru

Trochę zmodyfikowaliśmy oryginalny zbiór, więc zapiszmy go do łatwiejszego użycia w następnych rozdziałach.

In [15]:
credit_cards_df.to_parquet("../data/credit-cards-preprocessed.parquet")